### Se carga el archivo de Excel y se visualizan las hojas

In [4]:
import pandas as pd

# Cargar el archivo Excel
file_path = '../data/Internet.xlsx'
excel_data = pd.ExcelFile(file_path)

# Ver las hojas del archivo
print(excel_data.sheet_names)


['Acc_vel_loc_sinrangos', 'Velocidad_sin_Rangos', 'Accesos_tecnologia_localidad', 'Velocidad % por prov', 'Totales VMD', 'Totales Accesos Por Tecnología', 'Accesos Por Tecnología', 'Penetración-poblacion', 'Penetracion-hogares', 'Penetracion-totales', 'Totales Accesos por velocidad', 'Accesos por velocidad', 'Dial-BAf', 'Totales Dial-BAf', 'Ingresos ']


##### Dado que el objetivo es realizar un análisis enfocado en el acceso a internet y otros servicios de comunicación, voy a seleccionar solo aquellas hojas de la base de datos que sean pertinentes para ese enfoque, omitiendo aquellas que no contribuyan directamente a los indicadores de acceso a internet y otros servicios de telecomunicaciones.

##### Las hojas más relevantes para este análisis inicial serán:

* ##### Accesos_tecnologia_localidad: Contiene información desglosada por localidad, que incluye datos sobre accesos a internet por diferentes tecnologías.
* ##### Penetración-poblacion: Esta hoja está relacionada con la penetración de internet en la población, un dato clave para medir el acceso.
* ##### Penetracion-hogares: Similar a la anterior, pero enfocada en la penetración del servicio de internet en los hogares, lo que es fundamental para calcular KPIs de crecimiento en accesos.
* ##### Accesos por velocidad: Nos puede dar información útil sobre la calidad del servicio a través de la distribución de la velocidad de acceso a internet.
* ##### Totales Accesos Por Tecnología: Puede proporcionar una visión general de los accesos a internet según las tecnologías usadas, útil para observar tendencias de crecimiento o saturación por tecnología.

### Cargar hojas específicas

In [7]:
sheets_to_load = [
    'Accesos_tecnologia_localidad',
    'Penetración-poblacion',
    'Penetracion-hogares',
    'Accesos por velocidad',
    'Totales Accesos Por Tecnología'
]

data_dict = {sheet: pd.read_excel(excel_data, sheet_name=sheet) for sheet in sheets_to_load}
print (data_dict)

{'Accesos_tecnologia_localidad':          Provincia     Partido               Localidad Link Indec  ADSL  \
0     BUENOS AIRES  25 de Mayo              25 de Mayo    6854100   755   
1     BUENOS AIRES  25 de Mayo               Del Valle    6854020   181   
2     BUENOS AIRES  25 de Mayo       Gobernador Ugarte    6854040     0   
3     BUENOS AIRES  25 de Mayo  Norberto de la Riestra    6854060     0   
4     BUENOS AIRES  25 de Mayo        Lucas Monteverde    6854050     0   
...            ...         ...                     ...        ...   ...   
3347           NaN         NaN                     NaN        NaN     0   
3348           NaN         NaN                     NaN        NaN     0   
3349           NaN         NaN                     NaN        NaN     0   
3350           NaN         NaN                     NaN        NaN     0   
3351           NaN         NaN                     NaN        NaN     0   

      CABLEMODEM  DIAL UP  FIBRA OPTICA  OTROS  SATELITAL  WIMAX  

### Verificación de valores faltantes y duplicados

In [8]:
# Función para revisar valores faltantes y duplicados
def check_data_integrity(df, sheet_name):
    missing_values = df.isnull().sum().sum()
    duplicate_rows = df.duplicated().sum()

    if missing_values > 0:
        print(f"Hay {missing_values} valores faltantes en la hoja '{sheet_name}'.")
    else:
        print(f"No hay valores faltantes en la hoja '{sheet_name}'.")

    if duplicate_rows > 0:
        print(f"Hay {duplicate_rows} filas duplicadas en la hoja '{sheet_name}'.")
    else:
        print(f"No hay filas duplicadas en la hoja '{sheet_name}'.")

# Aplicar la verificación a cada hoja
for sheet, df in data_dict.items():
    print(f"Revisando la hoja: {sheet}")
    check_data_integrity(df, sheet)

Revisando la hoja: Accesos_tecnologia_localidad
Hay 1120 valores faltantes en la hoja 'Accesos_tecnologia_localidad'.
Hay 279 filas duplicadas en la hoja 'Accesos_tecnologia_localidad'.
Revisando la hoja: Penetración-poblacion
No hay valores faltantes en la hoja 'Penetración-poblacion'.
No hay filas duplicadas en la hoja 'Penetración-poblacion'.
Revisando la hoja: Penetracion-hogares
No hay valores faltantes en la hoja 'Penetracion-hogares'.
No hay filas duplicadas en la hoja 'Penetracion-hogares'.
Revisando la hoja: Accesos por velocidad
Hay 6 valores faltantes en la hoja 'Accesos por velocidad'.
No hay filas duplicadas en la hoja 'Accesos por velocidad'.
Revisando la hoja: Totales Accesos Por Tecnología
No hay valores faltantes en la hoja 'Totales Accesos Por Tecnología'.
No hay filas duplicadas en la hoja 'Totales Accesos Por Tecnología'.


### Limpieza de duplicados y valores faltantes

* ### Accesos_tecnologia_localidad

In [9]:
# Limpiar duplicados en 'Accesos_tecnologia_localidad'
data_dict['Accesos_tecnologia_localidad_cleaned'] = data_dict['Accesos_tecnologia_localidad'].drop_duplicates()

# Verificar que ya no haya filas duplicadas
duplicate_rows_after_cleaning = data_dict['Accesos_tecnologia_localidad_cleaned'].duplicated().sum()
print(f"Filas duplicadas después de la limpieza: {duplicate_rows_after_cleaning}")

# Limpiar valores faltantes en columnas clave
data_dict['Accesos_tecnologia_localidad_cleaned'] = data_dict['Accesos_tecnologia_localidad_cleaned'].dropna(
    subset=['Provincia', 'Partido', 'Localidad', 'Link Indec'])

# Verificar que ya no haya valores faltantes en esas columnas
missing_values_after_cleaning = data_dict['Accesos_tecnologia_localidad_cleaned'].isnull().sum()
print(f"Valores faltantes después de la limpieza:\n{missing_values_after_cleaning}")


Filas duplicadas después de la limpieza: 0
Valores faltantes después de la limpieza:
Provincia        0
Partido          0
Localidad        0
Link Indec       0
ADSL             0
CABLEMODEM       0
DIAL UP          0
FIBRA OPTICA     0
OTROS            0
SATELITAL        0
WIMAX            0
WIRELESS         0
Total general    0
dtype: int64


* ### Accesos por velocidad

In [11]:
# Revisar valores faltantes en 'Accesos por velocidad'
missing_values_velocity = data_dict['Accesos por velocidad'].isnull().sum()
print("Valores faltantes por columna en 'Accesos por velocidad':\n", missing_values_velocity)

# Eliminar filas con valores faltantes en 'Accesos por velocidad'
data_dict['Accesos por velocidad_cleaned'] = data_dict['Accesos por velocidad'].dropna()

# Verificar que no haya valores faltantes después de la limpieza
missing_values_after_cleaning = data_dict['Accesos por velocidad_cleaned'].isnull().sum()
print("Valores faltantes después de la limpieza:\n", missing_values_after_cleaning)


Valores faltantes por columna en 'Accesos por velocidad':
 Año                    0
Trimestre              0
Provincia              0
HASTA 512 kbps         0
+ 512 Kbps - 1 Mbps    0
+ 1 Mbps - 6 Mbps      0
+ 6 Mbps - 10 Mbps     0
+ 10 Mbps - 20 Mbps    0
+ 20 Mbps - 30 Mbps    0
+ 30 Mbps              0
OTROS                  6
Total                  0
dtype: int64
Valores faltantes después de la limpieza:
 Año                    0
Trimestre              0
Provincia              0
HASTA 512 kbps         0
+ 512 Kbps - 1 Mbps    0
+ 1 Mbps - 6 Mbps      0
+ 6 Mbps - 10 Mbps     0
+ 10 Mbps - 20 Mbps    0
+ 20 Mbps - 30 Mbps    0
+ 30 Mbps              0
OTROS                  0
Total                  0
dtype: int64


### Identificación de outliers

#### Se utilizará la regla del rango intercuartílico (IQR), que define los outliers como aquellos puntos de datos que están 1.5 veces por debajo del primer cuartil o por encima del tercer cuartil.

* ### Accesos_tecnologia_localidad_cleaned

In [12]:
import numpy as np

# Función para identificar outliers usando el rango intercuartílico (IQR)
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Aplicar la función a las columnas relevantes de 'Accesos_tecnologia_localidad_cleaned'
columns_to_check = ['ADSL', 'CABLEMODEM', 'DIAL UP', 'FIBRA OPTICA', 'OTROS', 'SATELITAL', 'WIMAX', 'WIRELESS', 'Total general']

# Diccionario para almacenar los outliers detectados por columna
outliers_dict_accesos_tecnologia = {}

for column in columns_to_check:
    outliers = detect_outliers_iqr(data_dict['Accesos_tecnologia_localidad_cleaned'], column)
    outliers_dict_accesos_tecnologia[column] = outliers
    print(f"Outliers detectados en la columna {column}: {len(outliers)}")

# Visualizar algunos de los outliers en 'Total general'
outliers_dict_accesos_tecnologia['Total general'].head()


Outliers detectados en la columna ADSL: 593
Outliers detectados en la columna CABLEMODEM: 745
Outliers detectados en la columna DIAL UP: 374
Outliers detectados en la columna FIBRA OPTICA: 607
Outliers detectados en la columna OTROS: 640
Outliers detectados en la columna SATELITAL: 188
Outliers detectados en la columna WIMAX: 14
Outliers detectados en la columna WIRELESS: 409
Outliers detectados en la columna Total general: 508


,Provincia,Partido,Localidad,Link Indec,ADSL,CABLEMODEM,DIAL UP,FIBRA OPTICA,OTROS,SATELITAL,WIMAX,WIRELESS,Total general
0,BUENOS AIRES,25 de Mayo,25 de Mayo,6854100,755,4600,0,2,0,742,0,727,6826
8,BUENOS AIRES,9 de Julio,9 de Julio,6588100,4241,3553,80,6238,0,0,0,831,14943
15,BUENOS AIRES,Adolfo Alsina,Carhué,6007010,766,0,1,1417,1,0,0,1066,3251
28,BUENOS AIRES,Almirante Brown,Adrogué,6028010,86,9355,53,12087,91,1,0,26,21699
29,BUENOS AIRES,Almirante Brown,Burzaco,6028010,39,16766,35,12073,14,2,0,309,29238


* ### Penetración-poblacion

In [13]:
# Aplicar la función a las columnas relevantes de 'Penetración-poblacion'
columns_to_check = ['Accesos por cada 100 hab']

# Diccionario para almacenar los outliers detectados por columna
outliers_dict_penetracion_poblacion = {}

for column in columns_to_check:
    outliers = detect_outliers_iqr(data_dict['Penetración-poblacion'], column)
    outliers_dict_penetracion_poblacion[column] = outliers
    print(f"Outliers detectados en la columna {column}: {len(outliers)}")

# Visualizar algunos de los outliers en 'Accesos por cada 100 hab'
outliers_dict_penetracion_poblacion['Accesos por cada 100 hab'].head()


Outliers detectados en la columna Accesos por cada 100 hab: 41


,Año,Trimestre,Provincia,Accesos por cada 100 hab
1,2024,1,Capital Federal,48.78
25,2023,4,Capital Federal,47.80
49,2023,3,Capital Federal,47.60
73,2023,2,Capital Federal,47.27
97,2023,1,Capital Federal,50.23


* ### Penetracion-hogares 

In [14]:
# Aplicar la función a las columnas relevantes de 'Penetracion-hogares'
columns_to_check = ['Accesos por cada 100 hogares']

# Diccionario para almacenar los outliers detectados por columna
outliers_dict_penetracion_hogares = {}

for column in columns_to_check:
    outliers = detect_outliers_iqr(data_dict['Penetracion-hogares'], column)
    outliers_dict_penetracion_hogares[column] = outliers
    print(f"Outliers detectados en la columna {column}: {len(outliers)}")

# Visualizar algunos de los outliers en 'Accesos por cada 100 hogares'
outliers_dict_penetracion_hogares['Accesos por cada 100 hogares'].head()


Outliers detectados en la columna Accesos por cada 100 hogares: 20


,Año,Trimestre,Provincia,Accesos por cada 100 hogares
1,2024,1,Capital Federal,119.53
25,2023,4,Capital Federal,117.02
49,2023,3,Capital Federal,116.40
73,2023,2,Capital Federal,115.48
97,2023,1,Capital Federal,122.59


* ### Accesos por velocidad_cleaned

In [15]:
# Aplicar la función a las columnas relevantes de 'Accesos por velocidad_cleaned'
columns_to_check = ['HASTA 512 kbps', '+ 512 Kbps - 1 Mbps', '+ 1 Mbps - 6 Mbps',
    '+ 6 Mbps - 10 Mbps', '+ 10 Mbps - 20 Mbps', '+ 20 Mbps - 30 Mbps', '+ 30 Mbps', 'OTROS', 'Total']

# Diccionario para almacenar los outliers detectados por columna
outliers_dict_accesos_velocidad = {}

for column in columns_to_check:
    outliers = detect_outliers_iqr(data_dict['Accesos por velocidad_cleaned'], column)
    outliers_dict_accesos_velocidad[column] = outliers
    print(f"Outliers detectados en la columna {column}: {len(outliers)}")

# Visualizar algunos de los outliers en 'Total'
outliers_dict_accesos_velocidad['Total'].head()


Outliers detectados en la columna HASTA 512 kbps: 167
Outliers detectados en la columna + 512 Kbps - 1 Mbps: 116
Outliers detectados en la columna + 1 Mbps - 6 Mbps: 139
Outliers detectados en la columna + 6 Mbps - 10 Mbps: 111
Outliers detectados en la columna + 10 Mbps - 20 Mbps: 126
Outliers detectados en la columna + 20 Mbps - 30 Mbps: 125
Outliers detectados en la columna + 30 Mbps: 131
Outliers detectados en la columna OTROS: 115
Outliers detectados en la columna Total: 163


,Año,Trimestre,Provincia,HASTA 512 kbps,+ 512 Kbps - 1 Mbps,+ 1 Mbps - 6 Mbps,+ 6 Mbps - 10 Mbps,+ 10 Mbps - 20 Mbps,+ 20 Mbps - 30 Mbps,+ 30 Mbps,OTROS,Total
0,2024,1,Buenos Aires,26002.0,22510.0,221185.0,233225.0,241778.0,106418.0,4068292.0,125147.0,5044557.0
1,2024,1,Capital Federal,516.0,5418.0,29753.0,51415.0,37369.0,5957.0,1374283.0,0.0,1504711.0
5,2024,1,Córdoba,99.0,8056.0,110371.0,87798.0,47085.0,29939.0,784141.0,14710.0,1082199.0
20,2024,1,Santa Fe,468.0,6417.0,76294.0,153030.0,58595.0,28498.0,579118.0,16168.0,918588.0
24,2024,4,Buenos Aires,26909.0,23962.0,228393.0,239339.0,246144.0,110876.0,4042646.0,93351.0,5011620.0


* ### Totales Accesos Por Tecnología

In [16]:
# Aplicar la función a las columnas relevantes de 'Totales Accesos Por Tecnología'
columns_to_check = ['ADSL', 'Cablemodem', 'Fibra óptica', 'Wireless', 'Otros', 'Total']

# Diccionario para almacenar los outliers detectados por columna
outliers_dict_totales_accesos_tecnologia = {}

for column in columns_to_check:
    outliers = detect_outliers_iqr(data_dict['Totales Accesos Por Tecnología'], column)
    outliers_dict_totales_accesos_tecnologia[column] = outliers
    print(f"Outliers detectados en la columna {column}: {len(outliers)}")

# Visualizar algunos de los outliers en 'Fibra óptica'
outliers_dict_totales_accesos_tecnologia['Fibra óptica'].head()


Outliers detectados en la columna ADSL: 0
Outliers detectados en la columna Cablemodem: 0
Outliers detectados en la columna Fibra óptica: 3
Outliers detectados en la columna Wireless: 0
Outliers detectados en la columna Otros: 0
Outliers detectados en la columna Total: 0


,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total,Periodo
0,2024,1,774475,5986957,4015101,598682,257941,11633156,Ene-Mar 2024
1,2023,4,836390,6022532,3908183,585760,194796,11547661,Oct-Dic 2023
2,2023,3,897895,6018832,3708718,581436,200027,11406908,Jul-Sept 2023
